## Daily Challenge: NumPy, Pandas, and Matplotlib Integration

### 1. Data Preparation:

Hint 1: Use np.random.uniform(low, high, size) to generate the temperature data. \
Hint 2: Create a DataFrame using pd.DataFrame(data, index, columns) with appropriate index and columns. 

Use NumPy to generate a synthetic dataset representing average monthly temperatures (in degrees Celsius) for 12 months across 10 different cities. The temperatures should range from -5 to 35 degrees. 

Convert this NumPy array into a Pandas DataFrame, adding city names as index and months as columns.

In [1257]:
# !pip install geopy

In [1258]:
import numpy as np
import pandas as pd

#  Capitals of the world
capitals = ['Tel Aviv', 'London', 'Paris', 'Berlin', 'Tokyo', 'Beijing', 'Moscow', 'Washington', 'Ottawa']

months = pd.date_range(start='2024-01-01', end='2024-12-01', freq='MS').strftime('%B')

In [1259]:
from geopy.geocoders import Nominatim

# I used geopy to get latitude for futher temperature generation
gl = Nominatim(user_agent="di_challenge")
capitals_coord= [gl.geocode(city, language='en', timeout=100) for city in capitals]

capitals_coord

[Location(Tel-Aviv, Tel Aviv Subdistrict, Tel-Aviv District, Israel, (32.0852997, 34.7818064, 0.0)),
 Location(London, Greater London, England, United Kingdom, (51.4893335, -0.14405508452768728, 0.0)),
 Location(Paris, Ile-de-France, Metropolitan France, France, (48.8588897, 2.3200410217200766, 0.0)),
 Location(Berlin, Germany, (52.510885, 13.3989367, 0.0)),
 Location(Tokyo, Japan, (35.6821936, 139.762221, 0.0)),
 Location(Beijing, China, (40.190632, 116.412144, 0.0)),
 Location(Moscow, Central Federal District, Russia, (55.625578, 37.6063916, 0.0)),
 Location(Washington, United States, (47.2868352, -120.212613, 0.0)),
 Location(Ottawa, Eastern Ontario, Ontario, Canada, (45.4208777, -75.6901106, 0.0))]

In [1260]:
import numpy as np
import random

# I thought that it would be interesting to try to generate temperature close to the actual
# I decided to use latitude and month to account for the effect of seasonality and closeness to the equator
# But it turns out that it is not that easy to find a good formula
# Here is the best that I could come up with

def generate_temperature(month, latitude, base_temp=0, mon_adj=8, lat_adj=11, temp_range=2):
    # Assuming months close to Jan are the coldest and to July are the warmest
    # We can use cosine function so that factor would approach -1 the closer month is to Jan and 1 for July
    # We achieve this by scaling month so that period is 12 and then adding pi to shift the cosine function to the right
    # I've researched how temp changes monthly througt the year and found that this approach gives good results
    mon_scale = np.cos(((month - 1) * 2 * np.pi / 12) + np.pi)
    
    # Here we adjust for the latitude.
    # So it will be 1 the closer it to the equator and -1 the closer it to the poles
    lat_scale = np.cos(np.pi * abs(latitude) / 90)
    
    # So we use base temperature and adjust it for the month and latitude
    adj_temp = base_temp + mon_scale * mon_adj  + lat_scale * lat_adj
    
    return np.random.uniform(adj_temp - temp_range, adj_temp + temp_range)

In [1261]:
import plotly.graph_objects as go
import random
import plotly.colors as colors

# Here I tried to find best values for monthly and latitude adjustments
# But if i set up weather limit close to actual weather, it is impossible 
# to find corresponding values for mon_adj and lat_adj

months_num = list(range(1, 13))
fig = go.Figure()
plots = 0

for _ in range(1000):
    max_temp = 30
    mon_adj = random.randint(1, max_temp)
    lat_adj = random.randint(1, max_temp)
    tel_aviv_temps = [generate_temperature(i, 32, mon_adj, lat_adj) for i in months_num]
    moscow_temps = [generate_temperature(i, 52, mon_adj, lat_adj) for i in months_num]
    
    tel_aviv_max = np.max(tel_aviv_temps)
    tel_aviv_min = np.min(tel_aviv_temps)
    moscow_max = np.max(moscow_temps)
    moscow_min = np.min(moscow_temps)
    
    if tel_aviv_min > -5 and moscow_max > 10 and tel_aviv_max < 35 and moscow_min < 0:
        color = colors.sample_colorscale(colors.PLOTLY_SCALES['Rainbow'], random.random())[0]

        fig.add_trace(go.Scatter(x=months_num, y=tel_aviv_temps, mode='lines', 
                                name=f'Tel Aviv (32°) {mon_adj} {lat_adj} {tel_aviv_max:.2f} {tel_aviv_min:.2f}', line=dict(color=color, width=2)))
        fig.add_trace(go.Scatter(x=months_num, y=moscow_temps, mode='lines', 
                             name=f'Moscow (52°) {mon_adj} {lat_adj} {moscow_max:.2f} {moscow_min:.2f}', line=dict(color=color, dash='dash', width=2)))
        plots += 1
        if plots > 10:
            break
    
fig.update_layout(
    legend=dict(xanchor='left', yanchor='top'),
    width=1200,
    height=600,
)

fig.show()

# Good values
# 8 20
# 9 22
# 10 17
# 8 11


In [1262]:
help(capitals_coord[0])

Help on Location in module geopy.location object:

class Location(builtins.object)
 |  Location(address, point, raw)
 |
 |  Contains a parsed geocoder response. Can be iterated over as
 |  ``(location<String>, (latitude<float>, longitude<Float))``.
 |  Or one can access the properties ``address``, ``latitude``,
 |  ``longitude``, or ``raw``. The last
 |  is a dictionary of the geocoder's response for this item.
 |
 |  Methods defined here:
 |
 |  __eq__(self, other)
 |      Return self==value.
 |
 |  __getitem__(self, index)
 |      Backwards compatibility with geopy<0.98 tuples.
 |
 |  __getstate__(self)
 |      Helper for pickle.
 |
 |  __init__(self, address, point, raw)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  __iter__(self)
 |
 |  __len__(self)
 |
 |  __ne__(self, other)
 |      Return self!=value.
 |
 |  __repr__(self)
 |      Return repr(self).
 |
 |  __setstate__(self, state)
 |
 |  __str__(self)
 |      Return str(self).
 |
 |  ------------

In [1263]:
capitals_coord[0].raw


{'place_id': 374285331,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 1382494,
 'lat': '32.0852997',
 'lon': '34.7818064',
 'class': 'boundary',
 'type': 'administrative',
 'place_rank': 16,
 'importance': 0.6499881947160822,
 'addresstype': 'city',
 'name': 'Tel-Aviv',
 'display_name': 'Tel-Aviv, Tel Aviv Subdistrict, Tel-Aviv District, Israel',
 'boundingbox': ['32.0293437', '32.1469766', '34.7391310', '34.8522617']}

In [1264]:
months_df = pd.DataFrame({'month': months, 'mon_num': range(1, 13)})
months_df

,month,mon_num
0,January,1
1,February,2
2,March,3
3,April,4
4,May,5
5,June,6
6,July,7
7,August,8
8,September,9
9,October,10


In [1265]:
city_df = pd.DataFrame({
    'city': [city.raw['name'] for city in capitals_coord],
    'latitude': [city.latitude for city in capitals_coord],
    'longitude': [city.longitude for city in capitals_coord],
    'country': [city.raw['display_name'].split(',')[-1] for city in capitals_coord]
})

city_df



,city,latitude,longitude,country
0,Tel-Aviv,32.085300,34.781806,Israel
1,London,51.489334,-0.144055,United Kingdom
2,Paris,48.858890,2.320041,France
3,Berlin,52.510885,13.398937,Germany
4,Tokyo,35.682194,139.762221,Japan
5,Beijing,40.190632,116.412144,China
6,Moscow,55.625578,37.606392,Russia
7,Washington,47.286835,-120.212613,United States
8,Ottawa,45.420878,-75.690111,Canada


In [1266]:
city_temp = pd.merge(months_df, city_df, how='cross')

city_temp['temperature'] = city_temp.apply(lambda x: generate_temperature(x['mon_num'], x['latitude']), axis=1)

city_temp

,month,mon_num,city,latitude,longitude,country,temperature
0,January,1,Tel-Aviv,32.085300,34.781806,Israel,-2.476982
1,January,1,London,51.489334,-0.144055,United Kingdom,-9.417952
2,January,1,Paris,48.858890,2.320041,France,-9.336071
3,January,1,Berlin,52.510885,13.398937,Germany,-9.200915
4,January,1,Tokyo,35.682194,139.762221,Japan,-3.645597
...,...,...,...,...,...,...,...
103,December,12,Tokyo,35.682194,139.762221,Japan,-2.698174
104,December,12,Beijing,40.190632,116.412144,China,-3.827470
105,December,12,Moscow,55.625578,37.606392,Russia,-9.333523
106,December,12,Washington,47.286835,-120.212613,United States,-8.199901


In [1267]:
city_mon_temp = pd.pivot_table(city_temp, values='temperature', index=['city', 'country', 'latitude', 'longitude'], columns='month', aggfunc='first')

city_mon_temp = city_mon_temp.reset_index()
city_mon_temp = city_mon_temp.set_index('city')
city_mon_temp.columns = city_mon_temp.columns.values

city_mon_temp

,country,latitude,longitude,April,August,December,February,January,July,June,March,May,November,October,September
city,,,,,,,,,,,,,,,
Beijing,China,40.190632,116.412144,3.174105,7.372660,-3.827470,-3.670163,-6.172841,9.955631,7.413439,-0.598779,4.333658,-2.575694,1.660289,7.634195
Berlin,Germany,52.510885,13.398937,-4.485354,3.479256,-8.842996,-10.816701,-9.200915,6.269232,5.830473,-6.162276,1.501301,-5.078390,-1.035091,0.135086
London,United Kingdom,51.489334,-0.144055,-3.072178,6.122173,-7.769411,-9.935819,-9.417952,4.573928,3.836374,-4.698490,3.517860,-8.324724,-4.353192,-0.161731
Moscow,Russia,55.625578,37.606392,-4.246676,3.241165,-9.333523,-12.065838,-13.194758,3.703921,1.035776,-7.982611,-0.834694,-7.523729,-3.991322,-1.371775
Ottawa,Canada,45.420878,-75.690111,0.164935,7.946559,-7.600876,-8.376349,-6.933135,7.725352,5.042333,-5.357196,2.677773,-5.479742,-0.043262,2.413993
Paris,France,48.858890,2.320041,-0.170893,3.607206,-10.292447,-7.167650,-9.336071,5.823601,4.163301,-7.436644,3.483581,-5.650428,-3.014573,4.303305
Tel-Aviv,Israel,32.085300,34.781806,3.268228,10.462299,-0.596780,-2.674324,-2.476982,12.797251,12.936923,-0.594114,9.988766,2.082339,4.393573,10.367604
Tokyo,Japan,35.682194,139.762221,5.315523,9.721846,-2.698174,-2.278219,-3.645597,13.472659,8.948499,-0.106877,6.089017,1.012622,3.581652,8.992345
Washington,United States,47.286835,-120.212613,0.177238,5.069308,-8.199901,-7.352636,-10.497922,8.992161,4.390930,-5.512892,4.606305,-5.790065,-0.830845,2.556185


### 2. Data Analysis:

Hint 1: Calculate the annual average temperature using DataFrame.mean(axis). \
Hint 2: Find the city with the highest and lowest average temperature using idxmax() and idxmin() methods.

Calculate the annual average temperature for each city.

Identify the city with the highest and lowest average temperature for the year.

In [1268]:
cmt = city_mon_temp
cmt['avg_temp'] = cmt[months].mean(axis=1)
cmt

,country,latitude,longitude,April,August,December,February,January,July,June,March,May,November,October,September,avg_temp
city,,,,,,,,,,,,,,,,
Beijing,China,40.190632,116.412144,3.174105,7.372660,-3.827470,-3.670163,-6.172841,9.955631,7.413439,-0.598779,4.333658,-2.575694,1.660289,7.634195,2.058253
Berlin,Germany,52.510885,13.398937,-4.485354,3.479256,-8.842996,-10.816701,-9.200915,6.269232,5.830473,-6.162276,1.501301,-5.078390,-1.035091,0.135086,-2.367198
London,United Kingdom,51.489334,-0.144055,-3.072178,6.122173,-7.769411,-9.935819,-9.417952,4.573928,3.836374,-4.698490,3.517860,-8.324724,-4.353192,-0.161731,-2.473597
Moscow,Russia,55.625578,37.606392,-4.246676,3.241165,-9.333523,-12.065838,-13.194758,3.703921,1.035776,-7.982611,-0.834694,-7.523729,-3.991322,-1.371775,-4.380339
Ottawa,Canada,45.420878,-75.690111,0.164935,7.946559,-7.600876,-8.376349,-6.933135,7.725352,5.042333,-5.357196,2.677773,-5.479742,-0.043262,2.413993,-0.651635
Paris,France,48.858890,2.320041,-0.170893,3.607206,-10.292447,-7.167650,-9.336071,5.823601,4.163301,-7.436644,3.483581,-5.650428,-3.014573,4.303305,-1.807309
Tel-Aviv,Israel,32.085300,34.781806,3.268228,10.462299,-0.596780,-2.674324,-2.476982,12.797251,12.936923,-0.594114,9.988766,2.082339,4.393573,10.367604,4.996232
Tokyo,Japan,35.682194,139.762221,5.315523,9.721846,-2.698174,-2.278219,-3.645597,13.472659,8.948499,-0.106877,6.089017,1.012622,3.581652,8.992345,4.033775
Washington,United States,47.286835,-120.212613,0.177238,5.069308,-8.199901,-7.352636,-10.497922,8.992161,4.390930,-5.512892,4.606305,-5.790065,-0.830845,2.556185,-1.032678


In [1269]:
print(f'City with the highest average temperature of {cmt.loc[cmt.avg_temp.idxmax(), "avg_temp"]:.2f} '
      f'is {cmt.avg_temp.idxmax()}')
print(f'City with the lowest average temperature of {cmt.loc[cmt.avg_temp.idxmin(), "avg_temp"]:.2f} '
      f'is {cmt.avg_temp.idxmin()}')

City with the highest average temperature of 5.00 is Tel-Aviv
City with the lowest average temperature of -4.38 is Moscow


In [1271]:
import plotly.graph_objects as go
from ipywidgets import widgets
from IPython.display import display
import numpy as np


fig = go.FigureWidget(go.Figure())

city_val = widgets.Dropdown(
    description='City:',
    value='All',
    options=np.append('All', city_temp.city.unique())
)

def update_plot(change):
    city_name = city_val.value
    if city_name == 'All':
        temp_trends = city_temp
    else:
        temp_trends = city_temp[city_temp['city'] == city_name]

    fig.data = []

    for city in temp_trends.city.unique():
        temp_trends_city = temp_trends[temp_trends.city==city]
        fig.add_trace(go.Scatter(x=temp_trends_city.mon_num, y=temp_trends_city['temperature'], name=city, mode='lines', showlegend=True))
    
    # Add horizontal line for average temperature
    avg_temp = temp_trends['temperature'].mean()
    fig.add_trace(go.Scatter(x=[temp_trends.mon_num.min(), temp_trends.mon_num.max()], 
                             y=[avg_temp, avg_temp],
                             name='Average', 
                             mode='lines', 
                             line=dict(color='black', dash='dash')))
    fig.update_layout(
        title=f'Temperature Trends in {city_name if city_name != "All" else "All Cities"}',
        xaxis_title='Date',
        yaxis_title='Temperature',
        width=800, height=500
    )


city_val.observe(update_plot, names='value')

# display(date_interval, fig)
display(widgets.VBox([city_val, fig]))

update_plot(None)